In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

In [ ]:
def yahoo_prices(tickers):
    data = yf.download(tickers, start='2011-01-01')
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')    

In [ ]:
# Generate our indicators and buy#sell signals for each of our assets
# df: contains one company (symbol) of data
def RSIcalc(df):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df['Price_change'] = df['Adj Close'].pct_change()

    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=19).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=19).mean()
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))

    df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy' ] = 'Yes'
    df.loc[(df['Adj Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy' ] = 'No'

    return df
    
print('RSIcalc function defined....')

In [ ]:
# Extract Buy & Sell dates from our Asset Dataframe
# df: contains one company (symbol) of data
def get_signals(df):
    Buying_dates = []
    Selling_dates = []
    for i in range(len(df) - 11):
        if 'Yes' in df['Buy'].iloc[i]:
            Buying_dates.append(df.iloc[i+1].name)  # name is the index that contains the date
            for j in range(1,11):
                if df['RSI'].iloc[i + j] > 40:
                    Selling_dates.append(df.iloc[i + j + 1].name)
                    break # Break before we reach the tenth day
                elif j == 10:
                    Selling_dates.append(df.iloc[i + j + 1].name)
    return Buying_dates, Selling_dates  

print('get_signals function defined....')  


In [ ]:
tickers = grab_symbols()
all_assets = yahoo_prices(tickers)

# Very long to run, will create a large file on system, BE CAREFUL
#all_assets.to_csv('SP_500_daily_prices.csv')

# Utility function to load prices from our saved historical file
#sp500 = pd.read_csv('SP_500_daily_prices.csv', index_col='Date')
#sp500.loc[sp500.Symbol == 'TSLA']

print('All S&P Prices loaded/downloaded')

In [ ]:
# Generate a list of trades for one specific company (symbol) based on our buy/sell signals
def log_trades(asset_frame):
    symbol = asset_frame.iloc[0].Symbol
    buy, sell = get_signals(asset_frame)
    buy_dates = asset_frame.loc[buy].index.tolist()
    buy_prices = asset_frame.loc[buy].Open.values.tolist()
    sell_dates = asset_frame.loc[sell].index.tolist()
    sell_prices = asset_frame.loc[sell].Open.values.tolist()

    trades = pd.DataFrame(buy_dates, columns=['Buy_dates'])
    trades['Buy_price'] = buy_prices
    trades['Sell_date'] = sell_dates
    trades['Sell_price'] = sell_prices
    trades['Overlaping_trades'] = [same_sell_date for same_sell_date in trades['Sell_date'] == trades['Sell_date'].shift(1)]
    trades['Profit_amount'] = trades['Sell_price'] - trades['Buy_price']
    trades['Profit'] = [amount for amount in trades['Profit_amount'] > 0]
    trades['Symbol'] = symbol
    
    # Remove overlapping trades
    trades = trades.loc[trades.Overlaping_trades == False]
    
    # Move Symbol to first column position for easy reading :)
    first_column = trades.pop('Symbol')
    trades.insert(0, 'Symbol', first_column)
    return trades

print('log_trades function defined....')

In [ ]:
tickers.index('TSLA')

In [ ]:
# Test our functions for only one stock before applying our trade log to all our 503 assets
i = 445
data = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
frame = RSIcalc(data)
if frame is not None:
    trades = log_trades(frame)

trades

In [ ]:
# Run our analysis on all our S&P500 companies
all_trades = None

len_tickers = len(tickers)
for i in range(len_tickers):
    data = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
    frame = RSIcalc(data)
    if frame is not None:
        trades = log_trades(frame)
    winning_trades = trades.Profit.sum()
    print(f'[{i}/{len_tickers}][{trades.iloc[0].Symbol}] Winning trades:{winning_trades}/{len(trades)} for total profit of ${trades.Profit_amount.sum():0.2f}')

    if all_trades is None:
        all_trades = trades.copy()
    else:
        all_trades = pd.concat([all_trades, trades], ignore_index=True).copy()

print(f'Total profit: ${all_trades.Profit_amount.sum():0.2f}')

In [ ]:
# Display statsitics on Winning vs Losing trades
# Winning trades : 5459
# Losing trades : 4194
# Total trades : 9653
win_trades = all_trades.loc[all_trades.Profit].Symbol.count()
win_amount = all_trades.loc[all_trades.Profit].Profit_amount.sum()
loss_trades = all_trades.loc[all_trades.Profit == False].Symbol.count()
loss_amount = all_trades.loc[all_trades.Profit == False].Profit_amount.sum()

print(f'Winners: {win_trades} for ${win_amount:0.2f} amount')
print(f'Loosers: {loss_trades} for ${loss_amount:0.2f} amount')
print(f'Profit: ${(win_amount + loss_amount):0.2f}')